In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T

In [348]:
VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

Files already downloaded and verified


In [3]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)

Conv2d       (8, 32, 32)    (3, 3)   (1, 1)  
BatchNorm2d  (8, 32, 32)   
LeakyReLU    (8, 32, 32)    (0.1)   
Conv2d       (192, 8, 8)    (1, 1)   (4, 4)  
ReLU         (192, 8, 8)   
BatchNorm2d  (192, 8, 8)   
Conv2d       (128, 2, 2)    (5, 5)   (2, 2)  
LeakyReLU    (128, 2, 2)    (0.1)   
MaxPool2d    (128, 1, 1)    (3, 3)   (3, 3)  
ReLU         (128, 1, 1)   
GlobalAvgPool2d (128, 1, 1)   
Flatten      128           
Linear       10            



In [350]:
fixed_model_gpu

Sequential(
  (0): Conv2d(3, 8, kernel_size=(6, 6), stride=(3, 3))
  (1): AvgPool2d(kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (2): PReLU(num_parameters=8)
  (3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (5): Conv2d(8, 16, kernel_size=(6, 6), stride=(3, 3), padding=(3, 3))
  (6): AvgPool2d(kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (7): ReLU()
  (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): AvgPool2d(kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (10): AvgPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0)
  (11): Flatten()
  (12): Linear(in_features=16, out_features=32, bias=True)
  (13): LeakyReLU(negative_slope=0.1)
  (14): Linear(in_features=32, out_features=10, bias=True)
)

In [351]:
loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

# express train
t, val_acc, train_acc = train_express_gpu(model = fixed_model_gpu,
                                          train = True,
                                          loader = loader_gpu,
                                          milestones = [],
                                          num_epoch = 1,
                                          verbose = True)

Finished in 0.4s 
val_acc: 0.209, train_acc: 0.198


In [10]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()

print(model_tmpl)

for t in model_tmpl.get_templates():
    print(t.type, t.get_learnable_params())

for t in model_tmpl.get_templates()[:-1]:
    print(encoder.encode(t))

Conv2d       (32, 10, 10)   (5, 5)   (3, 3)  
BatchNorm2d  (32, 10, 10)  
ReLU         (32, 10, 10)  
MaxPool2d    (32, 8, 8)     (3, 3)   (1, 1)  
AvgPool2d    (32, 9, 9)     (4, 4)   (1, 1)  
Conv2d       (192, 3, 3)    (5, 5)   (4, 4)  
ReLU         (192, 3, 3)   
BatchNorm2d  (192, 3, 3)   
MaxPool2d    (192, 1, 1)    (3, 3)   (3, 3)  
GlobalAvgPool2d (192, 1, 1)   
Flatten      192           
Linear       512           
PReLU        512           
Linear       64            
ReLU         64            
Linear       10            

Conv2dTmpl {'out_channels': 32, 'kernel_size': (5, 5), 'stride': (3, 3), 'padding': False}
BatchNorm2dTmpl {}
ReLUTmpl {}
MaxPool2dTmpl {'kernel_size': (3, 3), 'stride': (1, 1), 'padding': False}
AvgPool2dTmpl {'kernel_size': (4, 4), 'stride': (1, 1), 'padding': True, 'count_include_pad': False}
Conv2dTmpl {'out_channels': 192, 'kernel_size': (5, 5), 'stride': (4, 4), 'padding': True}
ReLUTmpl {}
BatchNorm2dTmpl {}
MaxPool2dTmpl {'kernel_size': (3, 3), '